In [94]:
import osmnx as ox
from osmnx import geocoder, features
import folium
import random
import networkx as nx
from osmnx import routing
from shapely.geometry import Point, LineString
place_name = "Hoan Kiem, Hanoi, Vietnam"
region = geocoder.geocode_to_gdf(place_name)
random.seed(42)

G = ox.graph_from_place(place_name, network_type="drive_service", simplify=False)
G = nx.subgraph(G, max(nx.strongly_connected_components(G), key=len)).copy()
nodes = list(G.nodes)
bounds = region.total_bounds
minx, miny, maxx, maxy = bounds
rand_lon =  105.85762
rand_lat = 21.02681
print(f"📍 Current location: lat={rand_lat:.5f}, lon={rand_lon:.5f}")

tags = {"amenity": "parking"}
gdf = features.features_from_place(place_name, tags=tags)

parking_points = []
orig_node = ox.distance.nearest_nodes(G, rand_lon, rand_lat)
nearest_park_node = None
min_dist = float("inf")
parking_nodes = [n for n, data in G.nodes(data=True) if data.get("amenity") == "parking"]
center_lat, center_lon = region.geometry.centroid.y.values[0], region.geometry.centroid.x.values[0]
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)
for _, row in gdf.iterrows():
    geom = row.geometry
    candidate_nodes = []

    if geom.geom_type == "Point":
        lat, lon = geom.y, geom.x
        parking_points.append((lat, lon))
        for n in parking_nodes:
            if (abs(G.nodes[n]['x'] - lon) < 1e-6) and (abs(G.nodes[n]['y'] - lat) < 1e-6):
                candidate_nodes.append(n)
        folium.CircleMarker( location=[lat, lon], radius=5, color="blue", fill=True, fill_color="blue", fill_opacity=0.8, popup="Parking spot", ).add_to(m)
    elif geom.geom_type in ["Polygon", "MultiPolygon"]:
        boundary_points = []
        if geom.geom_type == "Polygon":
            boundary_points = list(geom.exterior.coords)
        elif geom.geom_type == "MultiPolygon":
            for poly in geom.geoms:
                boundary_points.extend(list(poly.exterior.coords))

        for lon, lat in boundary_points[::max(1, len(boundary_points)//20)]:  
            candidate_nodes.append(ox.distance.nearest_nodes(G, lon, lat))

        folium.GeoJson(
            geom,
            style_function=lambda x: {"color": "blue", "fillOpacity": 0.3},
            tooltip="Parking area",
        ).add_to(m)
    for node in candidate_nodes:
        dist = nx.shortest_path_length(G, orig_node, node, weight="length")
        if dist < min_dist:
            min_dist = dist
            nearest_park_node = node
for lat, lon in parking_points:
    node = ox.distance.nearest_nodes(G, lon, lat)
    dist = nx.shortest_path_length(G, orig_node, node, weight="length")
    if dist < min_dist:
        min_dist = dist
        nearest_park_node = node

folium.Circle(
    location=[rand_lat, rand_lon],
    radius=5,  # very small inner circle
    color=None,
    fill=True,
    fill_color="red",
    fill_opacity=1
).add_to(m)

folium.Circle(
    location=[rand_lat, rand_lon],
    radius=10,  
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=0.2
).add_to(m)

route = nx.shortest_path(G, orig_node, nearest_park_node, weight="length")

route_gdf = routing.route_to_gdf(G, route, weight="length")

# add to folium
folium.GeoJson(
    route_gdf,
    style_function=lambda x: {"color": "blue", "weight": 4, "opacity": 0.8}
).add_to(m)

m.save("map_with_dijkstra.html")
print("✅ Map saved as map_with_dijkstra.html")


📍 Current location: lat=21.02681, lon=105.85762


C:\Users\Admin\AppData\Local\Temp\ipykernel_49528\1118855215.py:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




✅ Map saved as map_with_dijkstra.html


In [95]:
from IPython.display import display
display(m)